IMPORT LIBRARIES

In [24]:
import numpy as np
import cv2
import os
import pandas as pd
import string
import matplotlib.pyplot as plt

import os

from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler




In [28]:
alphabet = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
with open(r'/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/words.txt') as f:
    contents = f.readlines()

lines = [line.strip() for line in contents if line[0] in alphabet] 
lines[0]



'a01-000u-00-00 ok 154 408 768 27 51 AT A'

In [29]:
max_label_len = 0

char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz" 

# string.ascii_letters + string.digits (Chars & Digits)
# or 
# "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

print(char_list, len(char_list))

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, chara in enumerate(txt):
        dig_lst.append(char_list.index(chara))
        
    return dig_lst


!"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 78


In [30]:
images = []
labels = []

RECORDS_COUNT = 10000

train_images = []
train_labels = []
train_input_length = []
train_label_length = []
train_original_text = []

valid_images = []
valid_labels = []
valid_input_length = []
valid_label_length = []
valid_original_text = []

inputs_length = []
labels_length = []

In [31]:
def process_image(img):
    """
    Converts image to shape (32, 128, 1) & normalize
    """
    w, h = img.shape
    
#     _, img = cv2.threshold(img, 
#                            128, 
#                            255, 
#                            cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # Aspect Ratio Calculation
    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape
    
    img = img.astype('float32')
    
    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape
    
    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape
        
    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)
    
    img = cv2.subtract(255, img)
    
    img = np.expand_dims(img, axis=2)
    
    # Normalize 
    img = img / 255
    
    return img

In [34]:
for index, line in enumerate(lines):
    splits = line.split(' ')
    status = splits[1]
    
    if status == 'ok':
        word_id = splits[0]
        word = "".join(splits[8:])
        
        splits_id = word_id.split('-')
        filepath = r'/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/database/words\{}\{}-{}\{}.png'.format(splits_id[0], 
                                                  splits_id[0], 
                                                  splits_id[1], 
                                                  word_id)
        
        # processing on image
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        try:
            img = process_image(img)
        except:
            continue
            
        # processing on label
        try:
            label = encode_to_labels(word)
        except:
            continue
        
        if index % 10 == 0:
            valid_images.append(img)
            valid_labels.append(label)
            valid_input_length.append(31)
            valid_label_length.append(len(word))
            valid_original_text.append(word)
        else:
            train_images.append(img)
            train_labels.append(label)
            train_input_length.append(31)
            train_label_length.append(len(word))
            train_original_text.append(word)
        
        if len(word) > max_label_len:
            max_label_len = len(word)
    
    

    if index >= RECORDS_COUNT:
        break

[ WARN:0@525.327] global loadsave.cpp:244 findDecoder imread_('/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/database/words\a01\a01-000u\a01-000u-00-00.png'): can't open/read file: check file path/integrity
[ WARN:0@525.328] global loadsave.cpp:244 findDecoder imread_('/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/database/words\a01\a01-000u\a01-000u-00-01.png'): can't open/read file: check file path/integrity
[ WARN:0@525.328] global loadsave.cpp:244 findDecoder imread_('/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/database/words\a01\a01-000u\a01-000u-00-02.png'): can't open/read file: check file path/integrity
[ WARN:0@525.328] global loadsave.cpp:244 findDecoder imread_('/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/database/words\a01\a01-000

EXTRACT TEXT AND FILE NAME FROM WORD TEXT FILE

In [4]:
# d is the dictionary of key:file name, value:word text  
d = {}

# status of the word means that properly segmented or not. Two status - err:means word not segmented properly, ok:means word properly segmented.
# This contain only those word text which status is ok.
status = []

#key of dictionary which is file name
keys = []
with open('/home/ihsan/Documents/GitHub/ML-AI-ICT-course/Tcsion-Automate Extraction of handwritten text from an image/RIO_HandWrittenTextExtraction/FormsIAMSmall.txt') as f:
    for line in f:
        #split the line
        line = line.split(' ')
        status.append(line[1])
        if line[1] == 'ok':
            key = line[0]
            keys.append(line[0])
            text = line[8]
            text = text[:-1]
            d[key] = text
    f.close()
# Ground truth text. Actual word
gtText = []
for _, val in d.items():
    gtText.append(val)